In [1]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()
from scipy.io import loadmat
data = loadmat("letters_data.mat")
from sklearn.preprocessing import OneHotEncoder

In [2]:
train_x = data['train_x']

# y needs to be one hot encoded
old_y = data['train_y'] - 1


In [3]:
train_y = np.zeros((len(old_y), 26))
for i in range(len(old_y)):
    train_y[i, old_y[i]] = 1

In [4]:
shuffle = np.random.choice(np.arange(124800), replace=False, size=124800)
tx = train_x[shuffle][:100000]
ty = train_y[shuffle][:100000]
vx = train_x[shuffle][100000:]
vy = train_y[shuffle][100000:]

In [33]:
def batch_generator(n):
    while True:
        shuffle = np.random.choice(np.arange(100000), replace=False, size=100000)
        for i in range(1000):
            yield tx[i * n: (i+1) * n], ty[i * n: (i+1) * n]

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 26])

In [7]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [10]:
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [11]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [12]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [13]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [14]:
W_fc2 = weight_variable([1024, 26])
b_fc2 = bias_variable([26])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [34]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

batch_gen = batch_generator(100)
for i in range(20000):        
    batch = next(batch_gen)
    if i%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: vx, y_: vy, keep_prob: 1.0}))

step 0, training accuracy 0.01
step 10, training accuracy 0.05
step 20, training accuracy 0.13
step 30, training accuracy 0.24
step 40, training accuracy 0.34
step 50, training accuracy 0.42
step 60, training accuracy 0.41
step 70, training accuracy 0.43
step 80, training accuracy 0.4
step 90, training accuracy 0.55
step 100, training accuracy 0.55
step 110, training accuracy 0.58
step 120, training accuracy 0.54
step 130, training accuracy 0.61
step 140, training accuracy 0.53
step 150, training accuracy 0.62
step 160, training accuracy 0.7
step 170, training accuracy 0.5
step 180, training accuracy 0.67
step 190, training accuracy 0.66
step 200, training accuracy 0.68
step 210, training accuracy 0.66
step 220, training accuracy 0.59
step 230, training accuracy 0.51
step 240, training accuracy 0.6
step 250, training accuracy 0.57
step 260, training accuracy 0.62
step 270, training accuracy 0.73
step 280, training accuracy 0.63
step 290, training accuracy 0.62
step 300, training accura

KeyboardInterrupt: 